In [ ]:
# The Big 4
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Model performance metrics
from sklearn import metrics
from time import time

# Sklearn model tools
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score

# ensemble models
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier